In [1]:
from pathlib import Path

import pandas as pd
from neptoon.quality_assesment.quality_assesment import (
    QualityAssessmentFlagBuilder,
    FlagRangeCheck,
    FlagSpikeDetectionUniLOF,
)

from neptoon.data_management.crns_data_hub import CRNSDataHub

from neptoon.data_management.column_information import ColumnInfo
from neptoon.data_management.site_information import SiteInformation

from neptoon.neutron_correction.neutron_correction import (
    CorrectionType,
    CorrectionTheory,
)
from neptoon.calibration.station_calibration import CalibrationConfiguration
from neptoon.neutron_correction.correction_classes import Correction

from neptoon.data_management.data_audit import (
    DataAuditLog,
)
from neptoon.data_ingest_and_formatting.data_ingest import (
    ConfigurationManager
)
from neptoon.data_management.process_with_yaml import ProcessWithYaml

In [ ]:
DataAuditLog.create()

In [3]:
config = ConfigurationManager()
station_config_path = (
    Path.cwd() / "configuration_files" / "FSC001_station.yaml"
)
processing_config_path = (
    Path.cwd() / "configuration_files" / "v1_processing_method.yaml"
)

config.load_and_validate_configuration(
    name="station",
    file_path=station_config_path,
)
config.load_and_validate_configuration(
    name="processing",
    file_path=processing_config_path,
)

DataAuditLog.create()
yaml_processor = ProcessWithYaml(configuration_object=config)

## OPTION 1:
data_hub = yaml_processor.create_data_hub()

In [4]:
calib_df = pd.read_csv(
    Path.cwd() / "example_data" / "FSCD001_calibration.csv"
)
data_hub.calibration_samples_data = calib_df

The `attach_nmdb_data()` method has defaults. Running the below code, without selecting values, will work too. 


```python
data_hub.attach_nmdb_data()
```



In [ ]:
data_hub.attach_nmdb_data(
    station = "JUNG",
    new_column_name = "incoming_neutron_intensity",
    resolution = "60",
    nmdb_table = "revori"
    )
data_hub.crns_data_frame

In [ ]:
data_hub.prepare_static_values()
data_hub.crns_data_frame

In [ ]:
qa_flags = QualityAssessmentFlagBuilder()
qa_flags.add_check(
    FlagRangeCheck("air_relative_humidity", min_val=0, max_val=100),
    # FlagRangeCheck("precipitation", min_val=0, max_val=20),
    FlagSpikeDetectionUniLOF("epithermal_neutrons_cph"),
    # ...
)

data_hub.add_quality_flags(custom_flags=qa_flags)
data_hub.apply_quality_flags()
data_hub.flags_data_frame


In [ ]:
data_hub.select_correction(
    correction_type=CorrectionType.INCOMING_INTENSITY,
    correction_theory=CorrectionTheory.ZREDA_2012,
)
data_hub.select_correction(
    correction_type=CorrectionType.HUMIDITY,
    correction_theory=CorrectionTheory.ROSOLEM_2013
)

data_hub.select_correction(
    correction_type=CorrectionType.PRESSURE,
)

# data_hub.select_correction(
#     correction_type=CorrectionType.ABOVE_GROUND_BIOMASS
# )

data_hub.correct_neutrons()
data_hub.crns_data_frame

In [9]:
x = data_hub.crns_data_frame

In [ ]:
calibration_config = CalibrationConfiguration(
    date_time_column_name='DateTime_utc',
    profile_id_column='Profile_ID',
    distance_column='Distance_to_CRNS_m',
    sample_depth_column='Profile_Depth_cm',
    soil_moisture_gravimetric_column='SoilMoisture_g_g',
    bulk_density_of_sample_column='DryBulkDensity_g_cm3',
    soil_organic_carbon_column='SoilOrganicCarbon_g_g',
    lattice_water_column='LatticeWater_g_g',
)
data_hub.calibrate_station(config=calibration_config)
data_hub.calibrator.return_calibration_results_data_frame()

In [ ]:
data_hub.produce_soil_moisture_estimates()
data_hub.crns_data_frame

In [12]:
data_hub.save_data(append_yaml_hash_to_folder_name=True)

In [16]:
DataAuditLog.archive_and_delete_log(site_name='test')